In [1]:
import gymnasium as gym
from irbfn_mpc.nonlinear_mpc import NMPCPlanner
from irbfn_mpc.planner_utils import intersect_point, nearest_point_on_trajectory
import numpy as np

2024-06-24 18:08:11.392213: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
from IPython.display import HTML, display
%matplotlib inline

In [24]:
env = gym.make(
    "f1tenth_gym:f1tenth-v0",
    config={
        "observation_config": {"type": "kinematic_state"},
        "num_agents": 1,
        "control_input": "speed",
    },
    render_mode="rgb_array",
)
env = gym.wrappers.RecordVideo(env, "video_nmpc")

/usr/local/lib/python3.12/dist-packages/f1tenth_gym/envs/action.py:160: UserWarning: Only one control mode specified, using speed for longitudinal control and defaulting to steering angle for steering
  warnings.warn(


In [12]:
track = env.unwrapped.track
waypoints = np.stack([track.centerline.xs, track.centerline.ys, track.centerline.vxs, track.centerline.yaws]).T

In [13]:
mpc = NMPCPlanner()

In [6]:
def get_current_waypoint(waypoints, lookahead_distance, position, theta):
    wpts = waypoints[:, :2]
    lookahead_distance = np.float32(lookahead_distance)
    nearest_point, nearest_dist, t, i = nearest_point_on_trajectory(position, wpts)
    if nearest_dist < lookahead_distance:
        t1 = np.float32(i + t)
        lookahead_point, i2, t2 = intersect_point(
            position, lookahead_distance, wpts, t1, wrap=True
        )
        if i2 is None:
            return None, None
        current_waypoint = np.empty((3,), dtype=np.float32)
        # x, y
        current_waypoint[0:2] = wpts[i2, :]
        # speed
        current_waypoint[2] = waypoints[i, 2]
        return current_waypoint, i
    elif nearest_dist < 20:
        return wpts[i, :], i
    else:
        return None, None

In [26]:
obs, info = env.reset()
done = False

ld = 3.5
step = 0

while not done and step <= 4000:
    current_state = obs["agent_0"]
    goal, goal_i = get_current_waypoint(
        waypoints, ld, np.array([current_state["pose_x"], current_state["pose_y"]]), current_state["pose_theta"]
    )
    goal_state = np.array([goal[0], goal[1], 0.0, goal[2], waypoints[goal_i, 3]])
    accls, steer_vels = mpc.plan(current_state, goal_state)

    action = env.action_space.sample()
    action[0] = [steer_vels[0], accls[0]]
    # print(f"current state {current_state}")
    # print(f"goal state {goal_state}")
    # print(f"taking action {action[0]}")

    obs, step_reward, done, truncated, info = env.step(action)
    step += 1

env.close()

Moviepy - Building video /home/irbfn/scripts/video_nmpc/rl-video-episode-0.mp4.
Moviepy - Writing video /home/irbfn/scripts/video_nmpc/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/irbfn/scripts/video_nmpc/rl-video-episode-0.mp4


In [27]:
import glob
import io
import base64

for video_file in glob.glob("video_nmpc/*.mp4"):
    video = io.open(video_file, "rb").read()
    encoded = base64.b64encode(video).decode("ascii")
    display(
        HTML(
            f"""<video width="360" height="auto" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>"""
        )
    )